<center>
    <img src="https://github.com/juanandres-montero/macroeconometria/blob/main/logo%20EEC%20grande.png?raw=1" width="396" height="161"><br>
    <b style="font-size:200%">EC4301 MACROECONOMETRÍA</b><br>
    <b style="font-size:100%">Asistente: Juan Andrés Montero</b><br><br>
    <b style="font-size:100%"> Laboratorio #9:</b>
    <br>    
    <div style="font-size:150%; color:white; background-color: #0064b0; padding-top: 20px; padding-bottom: 20px; width: 50%; margin: 0 auto; border-radius: 15px; text-align: center;">
        Análisis de Cambio Estructural
    </div>
</center>

<br>

<i style="font-size:80%"> 
    Creado: Octubre 2024
</i>

<hr>

# Resumen del laboratorio anterior
## **Modelos ARCH y GARCH**
- Simulamos y estimamos modelos ARCH y GARCH en Stata
- Analizamos la heteroscedasticidad en series temporales
- Aplicamos estos modelos al análisis del tipo de cambio

# Análisis de Cambio Estructural en el Mercado de Gasolina

## **Contexto del Estudio**

Analizaremos el ejemplo de [Greene (2018)](#referencias) {cite:} sobre los datos del mercado de gasolina en Estados Unidos para el período 1953-2004. Este caso es particularmente interesante porque nos permite estudiar:

1. Las elasticidades ingreso y precio de la demanda
2. Los efectos de precios cruzados (autos nuevos y usados)
3. Un posible cambio estructural alrededor de 1973 (crisis del petróleo)

## **El Modelo Base**

Especificamos un modelo de demanda per cápita de gasolina:

ln(G/Pop) = β₁ + β₂ln(Income/Pop) + β₃lnPG + β₄lnPNC + β₅lnPUC + β₆t + ε

Donde:
- G/Pop: Consumo per cápita de gasolina
- Income/Pop: Ingreso per cápita
- PG: Índice de precios de la gasolina
- PNC: Precio de autos nuevos
- PUC: Precio de autos usados
- t: Tendencia temporal (Año - 1952)

## **1. La Prueba de Chow**

### **Fundamento Teórico**

La prueba de Chow nos permite evaluar si los parámetros del modelo son estables a lo largo del tiempo. En este caso, queremos probar si hubo un cambio estructural en 1973 debido a la crisis del petróleo.

El estadístico F se calcula como:

$$F[k, n_1 + n_2 - 2k] = \frac{(SSER - (SSE_1 + SSE_2)) / k}{(SSE_1 + SSE_2) / (n_1 + n_2 - 2k)}$$

Donde:
- SSER: Suma de residuos al cuadrado del modelo restringido (toda la muestra)
- SSE₁, SSE₂: Suma de residuos al cuadrado de cada submuestra
- k: Número de parámetros
- n₁, n₂: Tamaño de cada submuestra

### **Preparación y Análisis de Datos**

```stata
clear all
import excel "oil_test.xlsx", sheet("Hoja3") firstrow

* Conversión y generación de variables
destring GasExp Year Pop Gasp Income PNC PUC, replace
gen Gas_per_capita = GasExp/Pop/Gasp
gen t = Year-1952
gen lngpc = log(Gas_per_capita)
gen lnincome = log(Income)
gen lnpg = log(Gasp)
gen lnpnc = log(PNC)
gen lnpuc = log(PUC)

#### **Visualización**

```stata
gen cons_percapita_millon = (GasExp/Pop/Gasp)*1000000
twoway (lfitci Gasp cons_percapita_millon if Year > 1973, fcolor(green) lcolor(white) fintensity(20)) ///
       (scatter Gasp cons_percapita_millon if Year <= 1973, mcolor(red)) ///
       (scatter Gasp cons_percapita_millon if Year > 1973, mcolor(green)) ///
       (lfit Gasp cons_percapita_millon if Year <= 1973, lcolor(red) )  ///
       (lfit Gasp cons_percapita_millon if Year > 1973, lcolor(green) )  ///
       , legend(label(2) label(3 "1953-1973") label(4 "1974-2004") label(5 "Valores Ajustados") label(6 "Valores Ajustados")) ///
       xtitle("Consumo per cápita gasolina") ///
       ytitle("Índice de precios gasolina") ///
       title("Cambio estructural, test de Chow") ///
       text(100 4.2 "F(6,40) = 67.65 > 2.34", color(black)) ///
       text(120 4.2 "[(0.102 - 0.002 - 0.007)/6]/[(0.002 + 0.007)(52 - 2 x 6)]", color(black))
```

<center>
    <img src="https://github.com/juanandres-montero/macroeconometria/blob/main/assets/images/Lab9-graph1.png?raw=1" width="726" height="528"><br>
</center>

## **Estimación de Modelos**

### Modelo Completo (1953-2004):
```stata
regress lngpc lnincome lnpg lnpnc lnpuc t
```
<div style="display: flex; justify-content: center; margin: 20px;">
    <table style="border-collapse: collapse; width: auto; box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1); border-radius: 5px; overflow: hidden;">
        <thead>
            <tr style="background-color: #0064b0; color: white;">
                <th style="border: 1px solid #ddd; padding: 10px; text-align: left;">Coeficiente</th>
                <th style="border: 1px solid #ddd; padding: 10px; text-align: left;">Estimación</th>
            </tr>
        </thead>
        <tbody>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">Constante</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-26.6787</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(Income/Pop)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">1.6250</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(PG)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-0.05392</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(PNC)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-0.08343</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(PUC)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-0.08467</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">Tendencia</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-0.01393</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">R²</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.9649</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">Error estándar</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.04709</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;"><strong>Suma de cuadrados</strong></td>
                <td style="border: 1px solid #ddd; padding: 10px;"><strong>0.101997</strong></td>
            </tr>
        </tbody>
    </table>
</div>

### Primer Período (1953-1973):
```stata
regress lngpc lnincome lnpg lnpnc lnpuc t if Year<1974
```
<div style="display: flex; justify-content: center; margin: 20px;">
    <table style="border-collapse: collapse; width: auto; box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1); border-radius: 5px; overflow: hidden;">
        <thead>
            <tr style="background-color: #0064b0; color: white;">
                <th style="border: 1px solid #ddd; padding: 10px; text-align: left;">Coeficiente</th>
                <th style="border: 1px solid #ddd; padding: 10px; text-align: left;">Estimación</th>
            </tr>
        </thead>
        <tbody>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">Constante</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-22.1647</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(Income/Pop)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.8482</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(PG)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-0.03227</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(PNC)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.6988</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(PUC)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-0.2905</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">Tendencia</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.01006</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">R²</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.9975</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">Error estándar</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.01161</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;"><strong>Suma de cuadrados</strong></td>
                <td style="border: 1px solid #ddd; padding: 10px;"><strong>0.00202244</strong></td>
            </tr>
        </tbody>
    </table>
</div>


### Segundo Período (1974-2004):
```stata
regress lngpc lnincome lnpg lnpnc lnpuc t if Year>1973
```
<div style="display: flex; justify-content: center; margin: 20px;">
    <table style="border-collapse: collapse; width: auto; box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1); border-radius: 5px; overflow: hidden;">
        <thead>
            <tr style="background-color: #0064b0; color: white;">
                <th style="border: 1px solid #ddd; padding: 10px; text-align: left;">Coeficiente</th>
                <th style="border: 1px solid #ddd; padding: 10px; text-align: left;">Estimación</th>
            </tr>
        </thead>
        <tbody>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">Constante</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-15.3238</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(Income/Pop)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.3739</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(PG)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-0.1240</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(PNC)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-0.001146</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">ln(PUC)</td>
                <td style="border: 1px solid #ddd; padding: 10px;">-0.02167</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">Tendencia</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.004492</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">R²</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.9529</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;">Error estándar</td>
                <td style="border: 1px solid #ddd; padding: 10px;">0.01689</td>
            </tr>
            <tr style="transition: background-color 0.3s, transform 0.3s; cursor: pointer;" 
                onmouseover="this.style.backgroundColor='#00539f'; this.style.transform='scale(1.02)'" 
                onmouseout="this.style.backgroundColor='white'; this.style.transform='scale(1)'">
                <td style="border: 1px solid #ddd; padding: 10px;"><strong>Suma de cuadrados</strong></td>
                <td style="border: 1px solid #ddd; padding: 10px;"><strong>0.007127899</strong></td>
            </tr>
        </tbody>
    </table>
</div>


### **Prueba de Chow**

El estadístico F calculado es:

$$F[6,40] = \frac{(0.101997 - (0.002 + 0.007)) / 6}{(0.002 + 0.007) / 40} = 67.645$$

El valor crítico al 99% de confianza es 2.336, por lo que rechazamos la hipótesis nula de estabilidad estructural.

## Análisis de Resultados

1. **Elasticidad Ingreso**:
   - Pre-1973: 0.8482
   - Post-1973: 0.3739
   - La sensibilidad al ingreso disminuyó significativamente

2. **Elasticidad Precio de la Gasolina**:
   - Pre-1973: -0.03227
   - Post-1973: -0.1240
   - La demanda se volvió más sensible al precio

3. **Efectos de Precios Cruzados**:
   - Los coeficientes de PNC y PUC cambian sustancialmente
   - Sugiere un cambio en la relación entre mercados de autos y gasolina

## **Implicaciones Económicas**

1. La crisis del petróleo de 1973 marcó un cambio fundamental en el comportamiento del consumidor
2. La menor elasticidad ingreso post-1973 sugiere que la gasolina se convirtió en un bien más necesario
3. La mayor elasticidad precio indica una mayor sensibilidad a cambios en precios

## **2. Prueba de Andrews con Varianzas Desiguales**

La prueba de Chow asume que la varianza del error es constante en ambos períodos. Sin embargo, esto podría no ser cierto, especialmente en períodos de recesión económica.

### **Estadístico de Wald Robusto**

El estadístico de Wald se calcula como:

W = (β₁ - β₂)′(V₁ + V₂)⁻¹(β₁ - β₂)

Donde:
- β₁, β₂: Vectores de coeficientes estimados en cada período
- V₁, V₂: Matrices de covarianza de los estimadores

```stata
* Prueba de Andrews
estat sbsingle, trim(15) generate(Wald)
tsline Wald
```

# Referencias

Greene, W.H. (2018). *Econometric Analysis* (8th ed.). Pearson. ISBN: 9780134461366. [https://books.google.co.cr/books?id=xGZRvgAACAAJ](https://books.google.co.cr/books?id=xGZRvgAACAAJ)